GERADO PELO CHAT GPT

In [ ]:
!pip install PyPortfolioOpt

In [ ]:
import yfinance as yf
from pypfopt import risk_models, expected_returns
from pypfopt.black_litterman import BlackLittermanModel, market_implied_risk_aversion
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import numpy as np
import pandas as pd

# 1. Coletar dados históricos
tickers = ['PRIO3.SA','JHSF3.SA','ODPV3.SA','ITSA4.SA', 'EGIE3.SA' ,'LREN3.SA' ,'FLRY3.SA' ,'TRIS3.SA','SBFG3.SA',           'BBAS3.SA' ,'VALE3.SA' ,'ALOS3.SA','KLBN4.SA','CSAN3.SA','IVVB11.SA']

data = yf.download(tickers, start="2022-01-01", end="2023-12-31")['Close']
data = data.dropna()

# 2. Calcular retornos e matriz de covariância
returns = expected_returns.returns_from_prices(data)
S = risk_models.CovarianceShrinkage(data).ledoit_wolf()

# 3. Supor pesos do mercado (igual para simplificar)
market_weights = {ticker: 1/len(tickers) for ticker in tickers}

# 4. Definir opiniões
# Exemplo: "VALE3.SA terá 1% a mais de retorno que PETR4.SA"

#P = np.array([[0, 1, 0, 0],   # VALE3
#              [-1, 0, 0, 0]]) # -PETR4

P = np.zeros((1, len(tickers)))
P[0, tickers.index('VALE3.SA')] = 1
P[0, tickers.index('PRIO3.SA')] = -1

Q = np.array([0.01])          # Opinião: VALE3 - PETR4 = 1%
# Obs: Se quiser apenas uma linha para VALE3 > PETR4, P = [ -1, 1, 0, 0 ]

# 5. Construir o modelo Black-Litterman
view_confidences = np.array([0.75])  # 75% de confiança

bl = BlackLittermanModel(S,
                         pi=None,
                         market_caps=market_weights,
                         P=P[:1],  # selecionando só a primeira opinião para simplificar
                         Q=Q,
                         omega="idzorek",
                         view_confidences=view_confidences)  # define automaticamente a incerteza

bl_returns = bl.bl_returns()
bl_cov = bl.bl_cov()

# 6. Otimizar a carteira
ef = EfficientFrontier(bl_returns, bl_cov)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

# 7. Exibir resultados
df_pesos = pd.DataFrame(cleaned_weights.items(), columns=['Ticker', 'Peso'])
df_pesos['Perc_Peso'] = df_pesos['Peso'] * 100
print("\n🔎 Pesos do portfólio Black-Litterman:")
print(df_pesos.sort_values(by='Perc_Peso', ascending=False))


/tmp/ipython-input-8-196138270.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2022-01-01", end="2023-12-31")['Close']
[*********************100%***********************]  15 of 15 completed
/usr/local/lib/python3.11/dist-packages/pypfopt/black_litterman.py:258: UserWarning: Running Black-Litterman with no prior.
  warnings.warn("Running Black-Litterman with no prior.")


ValueError: at least one of the assets must have an expected return exceeding the risk-free rate